In [3]:
!pip install kokoro>=0.9.2
!pip install soundfile
!pip inatall google

ERROR: unknown command "inatall" - maybe you meant "install"



In [2]:
from google.colab import files
from kokoro import KPipeline
import json
import soundfile as sf
from IPython.display import display, Audio
import re

ModuleNotFoundError: No module named 'google.colab'

In [4]:
print("Please upload your JSON file containing the hieroglyph stories.")
uploaded = files.upload()
file_name = next(iter(uploaded))
with open(file_name, 'r', encoding='utf-8') as f:
    json_data = json.load(f)
print(f"Number of hieroglyphs: {len(json_data)}")

Please upload your JSON file containing the hieroglyph stories.


Saving hieroglyphs_stories (1).json to hieroglyphs_stories (1).json
Number of hieroglyphs: 179


In [5]:
class HieroglyphStoryReader:
    def __init__(self, json_data, lang_code='a'):
        self.data = json_data
        self.pipeline = KPipeline(lang_code=lang_code)

    def get_story(self, query):
        """Search for the story using the symbol or Gardiner number"""
        for key, value in self.data.items():
            if '-' in key:
                symbol, gardiner = key.split('-')
                if query == symbol or query == gardiner or query == key:
                    return value['story']
            else:
                if query == key:
                    return value['story']
        return "Story not found."

    def speak_story(self, query):
        """Play the story using Kokoro"""
        story = self.get_story(query)
        if story and story != "Story not found.":
            print(f" Speaking story: {story}")
            cleaned_story = self.clean_text(story)
            try:
                generator = self.pipeline(cleaned_story, voice='af_heart')
                for i, (gs, ps, audio) in enumerate(generator):
                    display(Audio(data=audio, rate=24000, autoplay=i==0))
                    sf.write(f'{i}.wav', audio, 24000)
            except Exception as e:
                print(f" Error during speech generation: {e}")
        else:
            print(" No story found for this symbol.")

    def clean_text(self, text):
        """Clean the text to improve pronunciation"""
        text = re.sub(r'[^\w\s.]', '', text)  # Keep letters, numbers, spaces, and periods
        text = re.sub(r'\s+', ' ', text)     # Reduce multiple spaces
        return text.strip()


# Initialize the reader
reader = HieroglyphStoryReader(json_data)

# Test the code
while True:
    user_input = input("Enter symbol, Gardiner number, or full key (or type 'exit'): ")
    if user_input.lower() == "exit":
        break
    reader.speak_story(user_input)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


kokoro-v1_0.pth:   0%|          | 0.00/327M [00:00<?, ?B/s]

Enter symbol, Gardiner number, or full key (or type 'exit'): 𓁟
 Speaking story: God with ibis head' refers to the ancient Egyptian god with the famous head symbolizing wisdom and knowledge. This description is used to refer to the god 'Thoth', who is often depicted with the head of an ibis. Thoth is considered the god of writing, sciences, and measurement, responsible for recording events and the calendar, highlighting his central role in ancient Egyptian culture.


af_heart.pt:   0%|          | 0.00/523k [00:00<?, ?B/s]

Enter symbol, Gardiner number, or full key (or type 'exit'): 𓋹
 Speaking story: An ankh, the key of life, symbolizing eternal life.might see it in the hands of gods or kings, showing their power over life and death.


Enter symbol, Gardiner number, or full key (or type 'exit'): exit
